In [ ]:
from json import *
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()
files=['task3_trainingset1_download_form.txt','task3_trainingset2_download_form.txt','task3_trainingset3_download_form.txt']
data={}
for filename in files:
    with open(filename,'r') as f:
        for x in f.readlines():
            if len(x)>1:
                #print(x)
                data[int(x.split('\t')[0])] =  int(x.replace('\n','').split('\t')[-1])
#print(data)
crawl={}

with open('tweet_by_ID_23_7_2018__03_55_00.txt','r',encoding='utf-8',errors='ignore') as f:
    for x in f.readlines():
        tp=JSONDecoder().decode(x)
        crawl[tp['id']]=tp['text'].lower()
#print(crawl)
alldata=[]
alllabel=[]
for i in crawl:
    if i in  data :
        alldata.append([wnl.lemmatize(x) for x in tknzr.tokenize(crawl[i])])
        alllabel.append(data[i])





In [ ]:
word_dict0={'PADDING':[0,99999]}
char_dict={'PADDING':0}
for i in alldata:
    
    for word in i:
        if word not in word_dict0:
            word_dict0[word]=[len(word_dict0),1]
        else:
            word_dict0[word][1]+=1
        for char in word:
            if char not in char_dict:
                char_dict[char]=len(char_dict)
        


In [ ]:
word_dict={'PADDING':[0,99999]}
for i in word_dict0:
    if word_dict0[i][1]>=2 and i not in word_dict:
        word_dict[i]=[len(word_dict),word_dict0[i][1]]
print(len(word_dict))

In [ ]:
import numpy as np
all_idword=[]

all_idchar=[]
for i in alldata:
    words=[x for x in i if x in word_dict]

    idword=[]
    idchar=[]
    
    
    for word in words:

        idword.append(word_dict[word][0])
        tp=[char_dict[m] for m in word][:15]
        idchar.append(tp+[0]*(15-len(tp)))
 
        
    idword=idword[:70]
    idchar=idchar[:70]

    all_idword.append(idword+[0]*(70-len(idword)))
    all_idchar.append(idchar+[[0]*15]*(70-len(idchar)))



In [ ]:
all_idword=np.array(all_idword,dtype='float32')
all_idchar=np.array(all_idchar,dtype='float32')
alllabel=np.array(alllabel,dtype='float32')


In [ ]:
import random
index=list(range(len(alllabel)))
random.shuffle(index)

In [ ]:
all_idword.shape

In [ ]:
nb=int(len(all_idword)*0.8)

In [ ]:
train_idword=np.array(all_idword,dtype='float32')[index[:nb]]
train_idchar=np.array(all_idchar,dtype='float32')[index[:nb]]
trainlabel=np.array(alllabel,dtype='float32')[index[:nb]]


dev_idword=np.array(all_idword,dtype='float32')[index[nb:]]
dev_idchar=np.array(all_idchar,dtype='float32')[index[nb:]]
devlabel=np.array(alllabel,dtype='float32')[index[nb:]]

In [ ]:

import numpy as np
embdict=dict()
plo=0
with open('/data1/wuch/word2vec_twitter_model.bin','rb')as f:
    header = f.readline()
    vocab_size, layer1_size = map(int, header.split())
    binary_len = np.dtype('float32').itemsize * layer1_size
    for line in range(vocab_size):
        word = []
        while True:
            ch = f.read(1).decode('utf-8','ignore')
            if ch ==' ':
                word = ''.join(word)
                break
            if ch != '\n':
                word.append(ch)
        if len(word) != 0:
            
            tp= np.fromstring(f.read(binary_len), dtype='float32')
            if word in word_dict:
                embdict[word]=tp.tolist()
                if plo %1000==0:
                    print(plo,line,word)
                plo+=1
                #print(word,tp)
        else:
            f.read(binary_len)
from numpy.linalg import cholesky
print(len(embdict),len(word_dict))
print(len(word_dict))
lister=[0]*len(word_dict)
xp=np.zeros(400,dtype='float32')

cand=[]
for i in embdict.keys():
    lister[word_dict[i][0]]=np.array(embdict[i],dtype='float32')
    cand.append(lister[word_dict[i][0]])
cand=np.array(cand,dtype='float32')

mu=np.mean(cand, axis=0)
Sigma=np.cov(cand.T)

norm=np.random.multivariate_normal(mu, Sigma, 1)
print(mu.shape,Sigma.shape,norm.shape)

for i in range(len(lister)):
    if type(lister[i])==int:
        lister[i]=np.reshape(norm, 400)
lister[0]=np.zeros(400,dtype='float32')
lister=np.array(lister,dtype='float32')
print(lister.shape)



In [ ]:

import tensorflow as tf

import sklearn
import numpy as np


In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

In [ ]:

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding, concatenate
from keras.layers import Dense, Input, Flatten, average,Lambda
from keras.layers import Convolution1D, MaxPooling1D,Conv1D,GlobalMaxPooling1D, AveragePooling1D, Embedding, Merge, Dropout, LSTM, GRU, Bidirectional, TimeDistributed
from keras.models import Model, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers #keras2
from keras.utils import plot_model
import numpy as np
from sklearn.metrics import *

from keras.layers import *

In [ ]:
#! -*- coding: utf-8 -*-

from keras import backend as K
from keras.engine.topology import Layer



class Attention(Layer):

    def __init__(self, nb_head, size_per_head, **kwargs):
        self.nb_head = nb_head
        self.size_per_head = size_per_head
        self.output_dim = nb_head*size_per_head
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.WQ = self.add_weight(name='WQ', 
                                  shape=(input_shape[0][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        self.WK = self.add_weight(name='WK', 
                                  shape=(input_shape[1][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        self.WV = self.add_weight(name='WV', 
                                  shape=(input_shape[2][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        super(Attention, self).build(input_shape)
        
    def Mask(self, inputs, seq_len, mode='mul'):
        if seq_len == None:
            return inputs
        else:
            mask = K.one_hot(seq_len[:,0], K.shape(inputs)[1])
            mask = 1 - K.cumsum(mask, 1)
            for _ in range(len(inputs.shape)-2):
                mask = K.expand_dims(mask, 2)
            if mode == 'mul':
                return inputs * mask
            if mode == 'add':
                return inputs - (1 - mask) * 1e12
                
    def call(self, x):
        #如果只传入Q_seq,K_seq,V_seq，那么就不做Mask
        #如果同时传入Q_seq,K_seq,V_seq,Q_len,V_len，那么对多余部分做Mask
        if len(x) == 3:
            Q_seq,K_seq,V_seq = x
            Q_len,V_len = None,None
        elif len(x) == 5:
            Q_seq,K_seq,V_seq,Q_len,V_len = x
        #对Q、K、V做线性变换
        Q_seq = K.dot(Q_seq, self.WQ)
        Q_seq = K.reshape(Q_seq, (-1, K.shape(Q_seq)[1], self.nb_head, self.size_per_head))
        Q_seq = K.permute_dimensions(Q_seq, (0,2,1,3))
        K_seq = K.dot(K_seq, self.WK)
        K_seq = K.reshape(K_seq, (-1, K.shape(K_seq)[1], self.nb_head, self.size_per_head))
        K_seq = K.permute_dimensions(K_seq, (0,2,1,3))
        V_seq = K.dot(V_seq, self.WV)
        V_seq = K.reshape(V_seq, (-1, K.shape(V_seq)[1], self.nb_head, self.size_per_head))
        V_seq = K.permute_dimensions(V_seq, (0,2,1,3))
        #计算内积，然后mask，然后softmax
        A = K.batch_dot(Q_seq, K_seq, axes=[3,3]) / self.size_per_head**0.5
        A = K.permute_dimensions(A, (0,3,2,1))
        A = self.Mask(A, V_len, 'add')
        A = K.permute_dimensions(A, (0,3,2,1))    
        A = K.softmax(A)
        #输出并mask
        O_seq = K.batch_dot(A, V_seq, axes=[3,2])
        O_seq = K.permute_dimensions(O_seq, (0,2,1,3))
        O_seq = K.reshape(O_seq, (-1, K.shape(O_seq)[1], self.output_dim))
        O_seq = self.Mask(O_seq, Q_len, 'mul')
        return O_seq
        
    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], input_shape[0][1], self.output_dim)

In [ ]:
MAX_SENT_LENGTH=70
MAX_CHAR_LENGTH=15
all_results2=[]
all_pred=[]
for k in range(5):
    pos_index=[]
    neg_index=[]
    for i in range(len(trainlabel)):
        if trainlabel[i]==1:
            pos_index.append(i)
        if trainlabel[i]==0:
            neg_index.append(i)
    new_pos_index=list(pos_index)*9+neg_index
    random.shuffle(new_pos_index)
    new_train_idword=np.array(train_idword,dtype='float32')[new_pos_index]
    new_train_idchar=np.array(train_idchar,dtype='float32')[new_pos_index]
    new_trainlabel=np.array(trainlabel,dtype='float32')[new_pos_index]

    embedding_layer = Embedding(len(word_dict),400,input_length=MAX_SENT_LENGTH,weights=[lister], trainable=True)
    
    
    review_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
    review_embedded_sequences = embedding_layer(review_input)
    d_review_emb=Dropout(0.2)(review_embedded_sequences)

    
    charembedding_layer = Embedding(len(char_dict),100,input_length=MAX_CHAR_LENGTH,trainable=True)
    
    char_input = Input(shape=(MAX_CHAR_LENGTH,), dtype='int32')
    char_embedded_sequences = charembedding_layer(char_input)
    d_char_emb=Dropout(0.2)(char_embedded_sequences)

    review_c_cnn_char = Convolution1D(nb_filter=200, filter_length=3, padding='same', activation='relu', strides=1)(d_char_emb)
    review_d_c_cnn_char = Dropout(0.2)(review_c_cnn_char)
    
    review_c_att0=Attention(16,16)([review_d_c_cnn_char,review_d_c_cnn_char,review_d_c_cnn_char])

    
    
    review_c_att = GlobalMaxPooling1D()(review_c_att0)
    charEncoder = Model(char_input, review_c_att)
    
    all_char_input = Input(shape=(MAX_SENT_LENGTH, MAX_CHAR_LENGTH), dtype='int32')
    char_encoder = TimeDistributed(charEncoder)(all_char_input)

    
    all_concat=concatenate([d_review_emb,char_encoder])
        

    
    review_c_cnn_sent = Convolution1D(nb_filter=200, filter_length=3, padding='same', activation='relu', strides=1)(all_concat)
    review_d_c_cnn_sent = Dropout(0.2)(review_c_cnn_sent)

    review_w_att0=Attention(16,16)([review_d_c_cnn_sent,review_d_c_cnn_sent,review_d_c_cnn_sent])
    
    review_w_att = GlobalMaxPooling1D()(review_w_att0)
    
    
    all_dense = Dense(200, activation='relu')(review_w_att)   
    classifier = Dense(2, activation='softmax')(all_dense)
    
    
    model = Model([review_input,all_char_input], classifier)
    
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])

    for i in range(1):
        model.fit([new_train_idword,new_train_idchar],to_categorical(new_trainlabel),shuffle=True,
                  class_weight='auto', batch_size=64, epochs=1,verbose=1)
        
        y_pred = model.predict([dev_idword,dev_idchar], batch_size=64, verbose=1)
        y_pred = np.argmax(y_pred, axis=1)
        y_true = devlabel
        cr = classification_report(y_true, y_pred,digits=4)
        print(accuracy_score(y_true, y_pred))
        print(cr)
        
        all_results2.append([precision_score(y_true, y_pred),recall_score(y_true, y_pred),f1_score(y_true, y_pred)])
   